# Try reading gnomAD/PanUKBB data into memory

In [1]:
using Revise
using EasyLD

┌ Info: Precompiling EasyLD [a5d00db5-5f0a-49b9-aaff-6f4121a13ee1]
└ @ Base loading.jl:1423
┌ Warning: Package EasyLD does not have LinearAlgebra in its dependencies:
│ - If you have EasyLD checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with EasyLD
└ Loading LinearAlgebra into EasyLD from project dependency, future warnings for EasyLD are suppressed.


In [2]:
datadir = "/Users/biona001/.julia/dev/EasyLD/data"
bm_file = joinpath(datadir, "UKBB.EUR.ldadj.bm")
ht_file = joinpath(datadir, "UKBB.EUR.ldadj.variant.ht")
bm = hail_block_matrix(bm_file, ht_file);

Size of matrix

In [3]:
size(bm)

2023-04-02 07:42:56.740 WARN  Utils:69 - Your hostname, Benjamins-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.0.0.86 instead (on interface en0)
2023-04-02 07:42:56.751 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address
2023-04-02 07:42:57.380 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Initializing Hail with default parameters...
Picked up _JAVA_OPTIONS: -Xmx16g
Picked up _JAVA_OPTIONS: -Xmx16g
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://10.0.0.86:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /Users/biona001/.julia/dev/EasyLD/test/hail-20230402-0742-0.2.109-b71b065e4bb6.log


(23960350, 23960350)

Read a single entry

In [4]:
bm[1, 1]

0.999959841549239

Read a block

In [5]:
@time bm[1:1000, 1:1000]

  3.477589 seconds (236.08 k allocations: 20.293 MiB, 9.16% compilation time)


1000×1000 Matrix{Float64}:
 0.99996  0.00098559  -0.000830543  …   0.00427643    0.000288337
 0.0      0.999942    -0.00104453      -0.000365344  -0.0014734
 0.0      0.0          0.999963        -0.000545369  -0.00101798
 0.0      0.0          0.0             -0.00052585   -0.00124045
 0.0      0.0          0.0              0.00195088    7.25693e-5
 0.0      0.0          0.0          …  -0.000118419  -0.000835871
 0.0      0.0          0.0             -9.25639e-5    0.00120466
 0.0      0.0          0.0             -0.000214606   0.000833805
 0.0      0.0          0.0             -0.000274162  -0.000390875
 0.0      0.0          0.0             -0.000110892   0.00294388
 0.0      0.0          0.0          …  -0.000373815  -0.000942022
 0.0      0.0          0.0             -0.000288357  -0.00193506
 0.0      0.0          0.0             -0.000162489  -0.00150939
 ⋮                                  ⋱                
 0.0      0.0          0.0             -0.000357731  -0.00113869
 0.0 

In [6]:
# indexing non-contiguous region is slow
@time bm[1:3, 1:2:100]

203.064163 seconds (111.13 k allocations: 5.909 MiB, 0.05% compilation time)


3×50 Matrix{Float64}:
 0.99996  -0.000830543  -0.000430697  …  -0.00995437   -0.0357603
 0.0      -0.00104453   -0.00056507       0.00112276   -0.0473314
 0.0       0.999963     -0.000921904     -0.000246667   0.0114384

Read a specific chromosome region

In [19]:
chr = 1 # string or Int
start_pos = 11063
end_pos = 1000000
sigma, df = get_block(bm, chr, start_pos, end_pos, min_maf=0.0, enforce_psd=true);
sigma

2444×2444 Matrix{Float64}:
  1.0           0.000985639  -0.000830575  …   0.0035368     0.000118066
  0.000985639   1.0          -0.00104458      -0.00103067   -0.000211577
 -0.000830575  -0.00104458    1.0             -0.000600475  -0.000372567
  0.000267975  -0.000939381  -0.00164298      -0.0016236    -0.000355651
 -0.000430709  -0.000565092  -0.000921929      0.434545     -0.00013721
  0.000132583  -0.000209847  -0.000379693  …  -0.000393288  -8.09227e-5
 -0.000158197  -0.000502146  -0.000874355      0.000181129  -0.000174117
 -0.000458895  -0.000332682  -0.000551908     -0.000624959  -0.000114182
  0.000221526  -0.000439541  -0.000786827      0.00212557   -0.000104047
  0.000979952  -0.000934722  -0.00154426      -0.000307705   0.00264703
  0.00305457   -0.000620115  -0.00107625   …  -0.00079625    0.010721
 -0.000427154  -0.000484291  -0.000863325      1.83475e-5   -0.000166744
 -0.000509855  -0.000409437  -0.000733328     -0.000173776  -0.000154475
  ⋮                           

In [21]:
df

Row,rsid,AF,chr,pos,ref,alt
,String,Float64,String3,Int64,String,String
1,rs561109771,4.7982e-5,1,11063,T,G
2,rs562993331,0.00027798,1,13259,G,A
3,rs578081284,0.00083096,1,17641,G,A
4,rs576081345,0.00065859,1,57222,T,C
5,rs570371753,0.00024023,1,58396,T,C
6,rs561430336,2.7728e-5,1,63668,G,A
7,rs2531267,0.00018542,1,69569,T,C
8,rs557418932,8.1599e-5,1,79192,T,G
9,rs554639997,0.00015868,1,91588,G,A


## Testing

In [44]:
using PyCall
py"""
from hail.linalg import BlockMatrix
import numpy as np
def convert(bm, range_start, range_end):
    return bm[range_start:range_end, range_start:range_end].to_numpy()
"""
@time py"convert"(bm.bm, 0, 100)

  0.827913 seconds (33.87 k allocations: 1.972 MiB, 1.93% compilation time)


100×100 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.0172358    -0.00857776
 0.0  1.0        -0.00041394  0.00540477     -0.0188801     0.05925
 0.0  0.0         1.0         0.0177243      -0.0277042    -0.000268512
 0.0  0.0         0.0         1.0            -0.000468342  -0.000966708
 0.0  0.0         0.0         0.0            -7.47553e-5   -0.00298802
 0.0  0.0         0.0         0.0         …   0.00207735    0.00152017
 0.0  0.0         0.0         0.0             0.00256425    0.00136302
 0.0  0.0         0.0         0.0             0.0137416    -0.000680087
 0.0  0.0         0.0         0.0            -0.020671      0.000342287
 0.0  0.0         0.0         0.0             0.00195264   -0.00451867
 0.0  0.0         0.0         0.0         …   0.0018285    -0.00406988
 0.0  0.0         0.0         0.0             0.0152337     0.0100539
 0.0  0.0         0.0         0.0             0.0193       -0.0136828
 ⋮                                        ⋱          

### Python code

In [ ]:
from hail.linalg import BlockMatrix
# import numpy as np
bm_files = "UKBB.EUR.ldadj.bm"
bm = BlockMatrix.read(bm_files)

# get a slice fast
x = bm[0:10000, 0:10000]
X = x.to_numpy()
X

In [ ]:
def convert(bm, range1, range2)
    return bm[range1, range2].to_numpy()